In [11]:
from langchain_community.llms import Ollama
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv, find_dotenv
import os
llm = Ollama(base_url="https://ollama.junzis.com", model="mixtral")
load_dotenv(find_dotenv())

True

In [12]:
tools = [TavilySearchResults(max_results=1, api_key=os.getenv("TAVILY_API_KEY"))]
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...


KeyboardInterrupt: 

In [10]:
llm.invoke("hi")

' Hello! How can I help you today? If you have any questions or need assistance with something related to programming, data science, machine learning, or artificial intelligence, feel free to ask. I will do my best to assist you.'